# Sparkify Spark Deployment
This project intends to use Sparkify user data to identify at-risk users who will likely churn.Full data (12GB) is used model training and hyperparameter tuning.

In [1]:
# Import libraries
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, col, concat, desc, lit, min, max, split, udf, countDistinct, sum, count
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql import Window

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression,GBTClassifier,LinearSVC, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1597553673190_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



In [2]:
# Create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load and Clean Dataset

In [3]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [4]:
# Print Schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [5]:
# Total sessions
print('Total activity records: ',df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total activity records:  26259199

In [6]:
# Filter out status Guest and Logged Out to exclude guests
df_clean = df.filter((col('auth') == 'Cancelled') | (col('auth') == 'Logged In'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Count User
df_clean.agg(countDistinct('userId')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                 22277|
+----------------------+

In [8]:
# Check for nulls
df_clean.filter(df_clean.userId == '').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [9]:
df_clean.filter(df_clean.sessionId == '').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

# Feature Engineering

In [10]:
# Get churn 1/0
cancel = udf(lambda auth: 1 if auth == 'Cancelled' else 0, IntegerType())
df_clean = df_clean.withColumn('cancel', cancel('auth'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Convert ts to date
df_feat = df_clean.withColumn("date", F.to_date(F.from_unixtime(col('ts')/lit(1000))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Get date range
df_feat.select('date').agg(min('date')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
| min(date)|
+----------+
|2018-10-01|
+----------+

In [13]:
df_feat.select('date').agg(max('date')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
| max(date)|
+----------+
|2018-12-01|
+----------+

In [14]:
# User registration date
df_feat = df_feat.withColumn('date_created', F.to_date(F.from_unixtime(col('registration')/lit(1000))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Get user age at the activity date
df_feat = df_feat.withColumn('user_age',F.datediff(col('date'),col('date_created')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df_feat.describe('user_age').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|          user_age|
+-------+------------------+
|  count|          25480720|
|   mean|  65.7917132639894|
| stddev|41.194336097140855|
|    min|               -23|
|    max|               411|
+-------+------------------+

In [17]:
# Get user interactions except for playing songs
inter = ['Thumbs Up', 'Thumbs Down', 'Add to Playlist', 'Add Friend']
get_interactions = udf(lambda x: 1 if x in inter else 0, IntegerType())

# Get negative feedback from user
thumbs_down = udf(lambda x: 1 if x == 'Thumbs Up' else 0, IntegerType())

df_feat = df_feat.withColumn('interactions', get_interactions('page'))
df_feat = df_feat.withColumn('thumbs_down', thumbs_down('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Get number of songs played
songs = udf(lambda x: 1 if x == 'NextSong' else 0, IntegerType())
df_feat = df_feat.withColumn('songs', songs('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Has the user ever used paied service
paid_hist = udf(lambda x: 1 if x == 'paid' else 0, IntegerType())
df_feat = df_feat.withColumn('paid_hist', paid_hist('level'))

# Has the user ever downgraded
downgrade = udf(lambda x: 1 if x == 'Submit Downgrade' else 0, IntegerType())
df_feat = df_feat.withColumn('downgraded', downgrade('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Convert gender to 1/0. 1 = female
female = udf(lambda x: 1 if x == 'F' else 0, IntegerType())
df_feat = df_feat.withColumn('female', female('gender'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Get user level dimensions
dims = df_feat.groupby('userId','female','date_created').agg(\
                                                          max('user_age').alias('user_age'),\
                                                          max('paid_hist').alias('paid_user'),\
                                                          max('downgraded').alias('downgraded'),\
                                                          max('cancel').alias('canceled'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Inspect Dimensions
dims.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(userId='1196880', female=1, date_created=datetime.date(2018, 9, 17), user_age=59, paid_user=1, downgraded=1, canceled=0)]

In [23]:
# Get measures based on usersessions and interactions
measures = df_feat.groupby('userId','sessionId').agg(\
                                                          min('date').alias('start_date'),\
                                                          max('date').alias('end_date'),\
                                                          sum('interactions').alias('interactions'),\
                                                          sum('thumbs_down').alias('thumbs_down'),\
                                                          sum('songs').alias('songs'),\
                                                          sum('length').alias('length'),\
                                                          countDistinct('artist').alias('c_artist'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Get the date gap between sessions
measures_lag = measures.withColumn('prev_end_date',
                        F.lag(col('end_date')).over(Window.partitionBy('userID').orderBy('start_date')))

measures_lag = measures_lag.withColumn('session_gap',\
                                      F.datediff(col('start_date'),col('prev_end_date')))

measures_agg = measures_lag.groupby('userId').agg(\
                                                 F.count('sessionId').alias('total_session'),\
                                                 avg('interactions').alias('interactions'),\
                                                 avg('thumbs_down').alias('thumbs_down'),\
                                                 avg('songs').alias('songs'),\
                                                 avg('length').alias('length'),\
                                                 avg('c_artist').alias('artists'),\
                                                 avg('session_gap').alias('session_gap'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Inspect measures
measures_agg.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(userId='1001393', total_session=7, interactions=11.142857142857142, thumbs_down=6.142857142857143, songs=70.57142857142857, length=17726.613365714282, artists=68.14285714285714, session_gap=9.5)]

In [26]:
# Inspect dim and measure shape
print('dim: ',dims.count(),' measure: ',measures_agg.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dim:  22277  measure:  22277

In [27]:
# Join dims and measures
output = dims.join(measures_agg, on = 'userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Modeling

In [30]:
# Assemble num features
assembler = VectorAssembler(inputCols=['female','user_age','paid_user',\
                                       'downgraded','total_session','interactions',\
                                      'thumbs_down','length','session_gap'],\
                            outputCol='NumFeatures',handleInvalid = 'skip')
data = assembler.transform(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Use 'canceled' as model prediction label
data = data.withColumnRenamed('canceled','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Get weighted col
balancingRatio = data.filter(col('label') == 1).count() / data.count()
calculateWeights = udf(lambda x: 1 * balancingRatio if x == 0 else (1 * (1.0 - balancingRatio)), DoubleType())

data = data.withColumn("classWeightCol", calculateWeights('label'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Split train test set
test, validation = data.randomSplit([0.8, 0.2], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Build pipeline
standardscaler = StandardScaler(inputCol="NumFeatures", outputCol="features", withMean=True, withStd=True)

lr = LogisticRegression(maxIter=10, regParam=0.0, weightCol = 'classWeightCol')
rf = RandomForestClassifier()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
def fit_model(model, paramGrid = None):
    pipeline = Pipeline(stages=[standardscaler, model])
    
    if paramGrid != None:
        crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)
        fitmodel = crossval.fit(test)
    else:
        fitmodel = pipeline.fit(test)
    
    results = fitmodel.transform(validation)
    
    return fitmodel, results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def val_evaluation(results):
    predictionAndLabels = results.select(['prediction', 'label']\
                                      ).withColumn('label',col('label').cast(DoubleType())).rdd

    metrics = MulticlassMetrics(predictionAndLabels)
    cm=metrics.confusionMatrix().toArray()
    
    accuracy=(cm[0][0]+cm[1][1])/cm.sum()
    precision=(cm[1][1])/(cm[0][1]+cm[1][1])
    recall=(cm[1][1])/(cm[1][0]+cm[1][1])
    return(round(accuracy,2),round(precision,2),round(recall,2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Logistic Regression Tuning

In [37]:
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1])\
    .addGrid(lr.threshold,[0.3,0.4,0.5])\
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
lrmodel, lrresults = fit_model(lr, paramGrid = lr_paramGrid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-38:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1744



In [39]:
bp_index = lrmodel.avgMetrics.index(np.max(lrmodel.avgMetrics))
bp = lr_paramGrid[bp_index]
print(bp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='LogisticRegression_b5d455972cbe', name='regParam', doc='regularization parameter (>= 0).'): 0.0, Param(parent='LogisticRegression_b5d455972cbe', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'): 0.5}

In [40]:
print("LogisticRegression: accuracy,precision,recall", val_evaluation(lrresults))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LogisticRegression: accuracy,precision,recall (0.72, 0.41, 0.78)

In [41]:
# Extract feature names from the original data
dict_feats = data.schema['NumFeatures'].metadata['ml_attr']['attrs']['numeric']
list_feats = np.array([x['name'] for x in dict_feats])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Get coefficients
lr_corr = lrmodel.bestModel.stages[-1].coefficients
print(list_feats, lr_corr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['female' 'user_age' 'paid_user' 'downgraded' 'total_session'
 'interactions' 'thumbs_down' 'length' 'session_gap'] [-0.01035194024658617,-0.511598612184427,0.17296282937871948,-0.050280973527757684,-0.5862069885608657,0.3034349717739996,-0.9266949112983791,0.47627147943738696,-1.7839078412107812]

### Random Forest Tuning

In [43]:
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth,[5,10,20])\
    .addGrid(rf.numTrees,[20,30])\
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
rfmodel, rfresults = fit_model(rf, paramGrid = rf_paramGrid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-44:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2736



In [45]:
brf_index = rfmodel.avgMetrics.index(np.max(rfmodel.avgMetrics))
brf = rf_paramGrid[brf_index]
print(brf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='RandomForestClassifier_dc88ae5333b9', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 20, Param(parent='RandomForestClassifier_dc88ae5333b9', name='numTrees', doc='Number of trees to train (>= 1).'): 20}

In [46]:
print("RandomForest: accuracy,precision,recall", val_evaluation(rfresults))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForest: accuracy,precision,recall (0.97, 0.97, 0.88)

In [47]:
# Extract feature importance from rfmodel
featImportances = np.array(rfmodel.bestModel.stages[-1].featureImportances)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
print(list_feats,featImportances)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['female' 'user_age' 'paid_user' 'downgraded' 'total_session'
 'interactions' 'thumbs_down' 'length' 'session_gap'] [0.02479987 0.25430633 0.00921502 0.01368088 0.19518738 0.07850133
 0.08206826 0.08112044 0.2611205 ]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…